## Overview:

This notebook processes and analyzes model test result JSON files to extract unique model responses, their scores and frequencies, and save the results for different context types. It includes functions to:

- Load and process JSON files for a given context and ID.
- Extract unique responses, their scores, and frequencies.
- Save the processed data for all context types into a single JSON file for a specified ID.

Additionally, it demonstrates saving the results for a specific ID as an example.

### Helper Functions


Load and Process JSON Files:

In [1]:
import os
import json
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import os
from groq import Groq



def __get_unique_model_responses(id: int, context_type: str, with_misleading: bool = False) -> list:
    """
    Load and process model test result JSON files for a given context and ID.

    Returns a list of unique model responses with their:
    - score (from first occurrence)
    - frequency (how many times that response appeared)
    
    Sorted by descending score.
    """
    model_name = 'llama-2-7b-80k'
    base_dir = Path('../results/graph')
    
    suffix = '_misleading' if with_misleading else ''
    directory = base_dir / f'{model_name}_id_{id}_{context_type}{suffix}'

    # Collect all responses and scores
    data = [
        json.load(open(directory / file, encoding='utf-8'))
        for file in os.listdir(directory) if file.endswith('.json')
    ]
    
    df = pd.DataFrame(data)[['model_response', 'needle', 'context_length', 'depth_percent']]
    #question_df = pd.read_excel("../data_generation/FreshQADataset_with_misleading_combined.xlsx")
    question_df = pd.read_json("../data_generation/context_cleaned.json")
    try:
        question = question_df.set_index("id").at[id, "question"]
        df['question'] = question
        
    except KeyError:
        raise ValueError(f"No question found for id: {id}")

    # Compute frequency
    freq = df['model_response'].value_counts().rename('frequency')

    # Drop duplicates but keep first score
    # df = df.drop_duplicates(subset='model_response')

    # Merge with frequency
    df = df.merge(freq, left_on='model_response', right_index=True)

    # Sort by score descending
    # df = df.sort_values(by='score', ascending=False).reset_index(drop=True)

    return df.to_dict(orient='records')


Save Processed Results:

In [2]:
def save_all_contexts_for_id(id: int) -> None:
    """
    Gather results from all 4 context types and save them in a single JSON file for the given id.

    File will be saved as: unique_responses/{id}.json
    """
    save_dir = Path('unique_responses')
    save_dir.mkdir(parents=True, exist_ok=True)

    all_data = {
        'relevant': __get_unique_model_responses(id, 'relevant', with_misleading=False),
        'relevant_misleading': __get_unique_model_responses(id, 'relevant', with_misleading=True),
        'irrelevant': __get_unique_model_responses(id, 'irrelevant', with_misleading=False),
        'irrelevant_misleading': __get_unique_model_responses(id, 'irrelevant', with_misleading=True),
    }

    save_path = save_dir / f'{id}.json'
    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(all_data, f, ensure_ascii=False, indent=2)

    print(f"Saved all context responses for ID {id} to {save_path}")

### Run Analysis:

In [3]:
# id_val = 44
ids = []
for file_name in os.listdir('../haystack/irrelevant'):
    if file_name.endswith('.txt'):
        file_id = os.path.splitext(file_name)[0]  # Extract the file name without extension
        ids.append(int(file_id))
print(ids)

[1, 11, 121, 122, 123, 124, 14, 15, 155, 16, 160, 163, 164, 167, 168, 17, 170, 172, 173, 175, 18, 180, 182, 183, 185, 189, 192, 193, 196, 198, 2, 29, 3, 380, 391, 393, 396, 4, 401, 403, 408, 409, 410, 411, 420, 421, 422, 423, 424, 427, 43, 430, 432, 433, 435, 436, 437, 438, 439, 44, 441, 442, 444, 448, 449, 453, 5, 532, 535, 539, 576, 577, 578, 579, 586, 587, 588, 589, 590, 92, 95, 96]


In [4]:
for i in ids:
    save_all_contexts_for_id(i)

Saved all context responses for ID 1 to unique_responses\1.json
Saved all context responses for ID 11 to unique_responses\11.json
Saved all context responses for ID 121 to unique_responses\121.json
Saved all context responses for ID 122 to unique_responses\122.json
Saved all context responses for ID 123 to unique_responses\123.json
Saved all context responses for ID 124 to unique_responses\124.json
Saved all context responses for ID 14 to unique_responses\14.json
Saved all context responses for ID 15 to unique_responses\15.json
Saved all context responses for ID 155 to unique_responses\155.json
Saved all context responses for ID 16 to unique_responses\16.json
Saved all context responses for ID 160 to unique_responses\160.json
Saved all context responses for ID 163 to unique_responses\163.json
Saved all context responses for ID 164 to unique_responses\164.json
Saved all context responses for ID 167 to unique_responses\167.json
Saved all context responses for ID 168 to unique_responses\1

In [5]:
# __get_unique_model_responses(ids, 'relevant', with_misleading=False)

In [6]:
def build_rewrite_prompt(model_response, real_needle, question):
    return (
        f"""You are evaluating a model-generated response for hallucination analysis in a long-context retrieval setting.

            You are given the following:

            Question: {question}

            The correct answer (real needle): {real_needle}

            A model-generated response: {model_response}

            Your task is to classify the model's response into one of the following categories:

            1. Correct Retrieval: the model answered the question correctly

            2. Excess Incorrect Retrieval: the model answered the question correctly but it also went on to output other text that was either completely irrelevant or invalid with regards to the question.

            3. Irrelevant Retrieval: the model did not answer the question, it went on to output other text

            4. Contradictory Retrieval: the model answered the question incorrectly

            Examples:

            Question: Who is the President of the United States? Needle: The President of the United States is Donald Trump.

            1: The President of the United States is Donald Trump. (Correct Retrieval)

            2: The President of the United States is Donald Trump. The affirmation was made in the Declaration of Independence, which was written predominantly by Thomas Jefferson in 1776. (Excess Incorrect Retrieval)

            3: The President of the United States is the head of state and head of government of the United States. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces. (Irrelevant Retrieval)

            4: Joe Biden is the President of the United States. (Contradictory Retrieval)

            Return your output in this exact JSON format: {{"model_response": "<text>", "needle": "<real_needle>", "hallucination_type": "<Correct Retrieval | Excess Incorrect Retrieval | Irrelevant Retrieval | Contradictory Retrieval>"}}"""
    )

In [7]:
# Load .env and API key
load_dotenv()
groq_api_key = os.getenv("snlp_api_key")
print(groq_api_key)

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


In [8]:
# # Initialize Groq client
# client = Groq(api_key=groq_api_key)

# def hallucination_categorising(response, needle, question):
#     while True:
#         try:
#             prompt = build_rewrite_prompt(response, needle, question)
#             completion = client.chat.completions.create(
#                 model="llama-3.3-70b-versatile",
#                 messages=[{"role": "user", "content": prompt}],
#                 temperature=0.7,
#                 max_tokens=512,
#                 top_p=1.0,
#                 stream=False,
#                 response_format={"type": "json_object"},
#                 stop=None,
#             )
#             content = completion.choices[0].message.content
#             data = json.loads(content)
#             return data["hallucination_type"]
#         except Exception as e:
#             print(f"Error: {e}")
#             print("Retrying now...")


In [9]:
from together import Together

client = Together(api_key="1fedf53fbab9628b7eae47f56d2bea3773754ece7206aa81a6faf83e73e11eb6")

def hallucination_categorising(response, needle, question):
    while True:
        try:
            prompt = build_rewrite_prompt(response, needle, question)
            completion = client.chat.completions.create(
                model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7,
                max_tokens=512,
                top_p=1.0,
                stream=False,
                response_format={"type": "json_object"},
                stop=None,
            )
            content = completion.choices[0].message.content
            data = json.loads(content)
            return data["hallucination_type"]
        except Exception as e:
            print(f"Error: {e}")
            print("Retrying now...")

In [10]:
lst_dfs = []
for i in ids:
    with open(f"unique_responses/{i}.json", "r", encoding="utf-8") as f:
        data = json.load(f)
        
    rows = []
    for category, entries in data.items():
        for entry in entries:
            entry["category"] = category
            rows.append(entry)

    df = pd.DataFrame(rows)
    # print(df)
    lst_dfs.append(df)


In [11]:
print(len(lst_dfs))

82


In [12]:
import time
from tqdm import tqdm

for single_df in lst_dfs:
    hallucination_types = []

    for idx, row in tqdm(single_df.iterrows(), total=len(single_df), desc="Categorising hallucinations"):
        start_time = time.time()  # ⏱ Track time at the start of each request

        mr = row.get("model_response", "")
        n = row.get("needle", "")
        q = row.get("question", "")

        current_hallucination_type = hallucination_categorising(mr, n, q)
        hallucination_types.append(current_hallucination_type)

        elapsed = time.time() - start_time
        sleep_time = max(0, 1.0 - elapsed)  # ⏸ Sleep the remaining time to enforce 1 req/sec
        time.sleep(sleep_time)

    single_df['hallucination_type'] = hallucination_types


Categorising hallucinations:  90%|█████████ | 90/100 [02:08<00:15,  1.53s/it]

Error: Expecting ':' delimiter: line 2437 column 1 (char 3002)
Retrying now...
Error: Expecting ',' delimiter: line 1 column 1161 (char 1160)
Retrying now...


Categorising hallucinations:  17%|█▋        | 17/100 [00:51<05:14,  3.79s/it]

Error: 'hallucination_type'
Retrying now...


Categorising hallucinations:  34%|███▍      | 34/100 [00:50<01:35,  1.44s/it]

...

Categorising hallucinations:  30%|███       | 30/100 [01:11<02:39,  2.28s/it]

Error: 'hallucination_type'
Retrying now...


Categorising hallucinations:  40%|████      | 40/100 [01:26<02:22,  2.38s/it]

Error: Expecting value: line 1 column 19 (char 18)
Retrying now...


Categorising hallucinations: 100%|██████████| 100/100 [02:29<00:00,  1.49s/it]


In [13]:
for df_single, id_single in zip(lst_dfs, ids):
    # Group by category and convert each group to a list of dicts
    json_output = {
        category: group.drop(columns="category").to_dict(orient="records")
        for category, group in df_single.groupby("category")
    }

    output_dir = "updated_unique_responses"
    file_id = str(id_single)  # or whatever variable you're using
    output_path = os.path.join(output_dir, f"{file_id}.json")

    # Create the folder if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Write the JSON file into the folder
    with open(output_path, "w") as f:
        json.dump(json_output, f, indent=4)
